In [1]:
import wmi
import gc
supported_cloudmusic_version = '2.10.6.3993'
wmic = wmi.WMI()
process = wmic.Win32_Process(name="cloudmusic.exe")
process = [p for p in process if '--type=' not in p.ole_object.CommandLine]
if not process:
    raise RuntimeError('No candidate process found')
elif len(process) != 1:
    raise RuntimeError('Multiple candidate processes found!')
else:
    process = process[0]
    from win32com.client import Dispatch
    ver_parser = Dispatch('Scripting.FileSystemObject')
    info = ver_parser.GetFileVersion(process.ExecutablePath)
    if info != supported_cloudmusic_version: raise RuntimeError(f'This version is not supported yet: {info}. Supported version: {supported_cloudmusic_version}')
    pid = process.ole_object.ProcessId
    del process
    del wmic
    del info
    del ver_parser
    del Dispatch
    gc.collect()

In [4]:
from pyMeow import open_process, get_module, r_float64, close_process, get_module
import psutil
current_offset = 0xA65568
maxlen_offset = 0xB16438  # TODO: does not work

process = open_process(pid)
ps_util_process = psutil.Process(pid)
base_address = get_module(process, 'cloudmusic.dll')['base']
maxlen = r_float64(process, base_address + maxlen_offset)
current = r_float64(process, base_address + current_offset)
close_process(process)
gc.collect()

0

In [5]:
import ctypes
from ctypes import wintypes

user32 = ctypes.windll.user32

WNDENUMPROC = ctypes.WINFUNCTYPE(
    wintypes.BOOL,
    wintypes.HWND,    # _In_ hWnd
    wintypes.LPARAM,) # _In_ lParam

user32.EnumWindows.argtypes = (
   WNDENUMPROC,      # _In_ lpEnumFunc
   wintypes.LPARAM,) # _In_ lParam

user32.IsWindowVisible.argtypes = (
    wintypes.HWND,) # _In_ hWnd

user32.GetWindowThreadProcessId.restype = wintypes.DWORD
user32.GetWindowThreadProcessId.argtypes = (
  wintypes.HWND,     # _In_      hWnd
  wintypes.LPDWORD,) # _Out_opt_ lpdwProcessId

user32.GetWindowTextLengthW.argtypes = (
   wintypes.HWND,) # _In_ hWnd

user32.GetWindowTextW.argtypes = (
    wintypes.HWND,   # _In_  hWnd
    wintypes.LPWSTR, # _Out_ lpString
    ctypes.c_int,)   # _In_  nMaxCount

def get_title(pid) -> str:
    ret = ''
    @WNDENUMPROC
    def enum_proc(hWnd, lParam):
        nonlocal ret
        if user32.IsWindowVisible(hWnd):
            _pid = wintypes.DWORD()
            user32.GetWindowThreadProcessId(hWnd, ctypes.byref(_pid))
            length = user32.GetWindowTextLengthW(hWnd) + 1
            title = ctypes.create_unicode_buffer(length)
            user32.GetWindowTextW(hWnd, title, length)
            if _pid.value == pid: ret = title.value
        return True
    user32.EnumWindows(enum_proc, 0)
    return ret

get_title(pid)

'星辰大海 - 黄霄雲'